In [163]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
wikipage="https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai"
results=requests.get(wikipage)
if results.status_code==200:
    soup=BeautifulSoup(results.content,"html.parser")
table=soup.find('table',{'class':'wikitable sortable'})
new_table=[]
for row in table.find_all('tr')[1:]:
    column_marker=0
    columns=row.find_all('td')
    new_table.append([column.get_text() for column in columns])
df=pd.DataFrame(new_table,columns=['Area','Location','Latitude','Longitude'])
df['Area']=df['Area'].str.replace('\n','')
df['Location']=df['Location'].str.replace('\n','')
df['Latitude']=df['Latitude'].str.replace('\n','')
df['Longitude']=df['Longitude'].str.replace('\n','')
df.shape
df.reset_index(drop=True)
df

,Area,Location,Latitude,Longitude
0,Amboli,"Andheri,Western Suburbs",19.1293,72.8434
1,"Chakala, Andheri",Western Suburbs,19.111388,72.860833
2,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373
3,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.82721
4,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.82927
...,...,...,...,...
88,Parel,South Mumbai,18.99,72.84
89,Gowalia Tank,"Tardeo,South Mumbai",18.96245,72.809703
90,Dava Bazaar,South Mumbai,18.946882,72.831362
91,Dharavi,Mumbai,19.040208,72.85085


In [7]:
pip install folium

     |████████████████████████████████| 93 kB 3.0 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [164]:
from geopy.geocoders import Nominatim
import folium
address='Mumbai'
geolocator=Nominatim(user_agent="capstone_IBM")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
map_mumbai=folium.Map(location=[latitude,longitude],zoom_start=11)
# add markers to the map 
for lat,long,ar,loc in zip(df['Latitude'],df['Longitude'],df['Area'],df['Location']):
    label='{} ,{}'.format(ar,loc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
     [lat,long],
     radius=7,
     popup=label,
     color='orange',
    fill=False,
    fill_opacity=0.8,     
      parse_html=False).add_to(map_mumbai) 
map_mumbai

Define Four Square Credentials

In [166]:
CLIENT_ID = '2LEU0L4R0EDCHYLHB2CPZPHCHXLULI3TT0PYIHRTE0XAWS4H' # your Foursquare ID
CLIENT_SECRET = 'QEFXXQ4JF214BY43A015LXZKMO2ZMWCMMSTVBLVUCWHECZOI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
def getnearbyVenues(names,latitudes,longitudes,radius=500):
    venues_list=[]
    for names,lat,long in zip(names,latitudes,longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            names, 
            lat, 
            long, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Location', 
                  'Location Latitude', 
                  'Location Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
M_venues = getnearbyVenues(names=df['Location'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
M_venues


,Location,Location Latitude,Location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Andheri,Western Suburbs",19.1293,72.8434,Cafe Arfa,19.128930,72.847140,Indian Restaurant
1,"Andheri,Western Suburbs",19.1293,72.8434,"5 Spice , Bandra",19.130421,72.847206,Chinese Restaurant
2,"Andheri,Western Suburbs",19.1293,72.8434,Subway,19.127860,72.844461,Sandwich Place
3,"Andheri,Western Suburbs",19.1293,72.8434,Cafe Coffee Day,19.127748,72.844663,Coffee Shop
4,"Andheri,Western Suburbs",19.1293,72.8434,Spices & Chillies,19.127765,72.844131,Asian Restaurant
...,...,...,...,...,...,...,...
1308,Mumbai,19.2,72.97,Borivali Biryani Centre,19.201597,72.973558,Indian Restaurant
1309,Mumbai,19.2,72.97,Makhmali Talao,19.198472,72.971438,Lake
1310,Mumbai,19.2,72.97,Royal Challenge II,19.203504,72.967853,Indian Restaurant
1311,Mumbai,19.2,72.97,Mochi,19.203317,72.967420,Shoe Store


In [167]:
restaurants=['Indian Restaurant','Asian Restaurant']
MI_Venues=M_venues[M_venues['Venue Category'].isin(restaurants)]
MI_Venues.reset_index()
MI_Venues

,Location,Location Latitude,Location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Andheri,Western Suburbs",19.1293,72.8434,Cafe Arfa,19.128930,72.847140,Indian Restaurant
4,"Andheri,Western Suburbs",19.1293,72.8434,Spices & Chillies,19.127765,72.844131,Asian Restaurant
12,Western Suburbs,19.111388,72.860833,The Mirador Mumbai,19.111462,72.860667,Asian Restaurant
25,Western Suburbs,19.111388,72.860833,HUL Canteen,19.113199,72.856825,Indian Restaurant
31,"Andheri,Western Suburbs",19.124085,72.831373,Shalimar Restaurant,19.127657,72.832669,Indian Restaurant
...,...,...,...,...,...,...,...
1295,South Mumbai,18.946882,72.831362,Bawarchi,18.946075,72.831738,Indian Restaurant
1296,South Mumbai,18.946882,72.831362,Jaffar Bhai Delhi Darbar,18.950000,72.834625,Indian Restaurant
1301,South Mumbai,18.946882,72.831362,Hotel Geeta Bhavan,18.945818,72.828304,Indian Restaurant
1308,Mumbai,19.2,72.97,Borivali Biryani Centre,19.201597,72.973558,Indian Restaurant


The Indian and Asian Restaurants of Mumbai

In [33]:
from geopy.geocoders import Nominatim
import folium
address='Mumbai'
geolocator=Nominatim(user_agent="capstone_IBM")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
map_mumbai_restaurants=folium.Map(location=[latitude,longitude],zoom_start=11)
# add markers to the map 
for lat,long,cat,loc in zip(MI_Venues['Venue Latitude'],MI_Venues['Venue Longitude'],MI_Venues['Venue Category'],MI_Venues['Location']):
    label='{} ,{}'.format(cat,loc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
     [lat,long],
     radius=7,
     popup=label,
     color='yellow',
    fill=False,
    fill_opacity=0.8,     
      parse_html=False).add_to(map_mumbai_restaurants) 
map_mumbai_restaurants

In order to apply KNN Algorithm convert the categorical variables ..

In [168]:
mumbai_restaurants_dummies = pd.get_dummies(MI_Venues[['Venue Category']], prefix="", prefix_sep="")
mumbai_restaurants_dummies['Location']=MI_Venues['Location']
fixed_columns = [mumbai_restaurants_dummies.columns[-1]] + list(mumbai_restaurants_dummies.columns[:-1])
mumbai_restaurants_dummies=mumbai_restaurants_dummies[fixed_columns]
mumbai_restaurants_dummies

,Location,Asian Restaurant,Indian Restaurant
0,"Andheri,Western Suburbs",0,1
4,"Andheri,Western Suburbs",1,0
12,Western Suburbs,1,0
25,Western Suburbs,0,1
31,"Andheri,Western Suburbs",0,1
...,...,...,...
1295,South Mumbai,0,1
1296,South Mumbai,0,1
1301,South Mumbai,0,1
1308,Mumbai,0,1


In [169]:
m_restaurants=mumbai_restaurants_dummies.groupby('Location').mean().reset_index(drop=False)
mum_rest=m_restaurants.drop('Location',1)
m_restaurants

,Location,Asian Restaurant,Indian Restaurant
0,"Andheri,Western Suburbs",0.250000,0.750000
1,"Bandra,Western Suburbs",0.222222,0.777778
2,"Borivali (West),Western Suburbs",0.000000,1.000000
3,"Byculla,South Mumbai",0.000000,1.000000
4,Eastern Suburbs,0.000000,1.000000
5,"Ghatkopar,Eastern Suburbs",0.133333,0.866667
6,"Govandi,Harbour Suburbs",0.000000,1.000000
7,Harbour Suburbs,0.000000,1.000000
8,"Kalbadevi,South Mumbai",0.000000,1.000000
9,"Kamathipura,South Mumbai",0.250000,0.750000


In [173]:
from sklearn.cluster import KMeans
kclusters = 6
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_rest)
kmeans.labels_[0:22] 
mum_rest.insert(0,'Cluster Labels',kmeans.labels_)
mum_rest['Cluster Labels']=mum_rest['Cluster Labels'].astype('str')
mum_rest.insert(3,"Location",m_restaurants['Location'])
mum_rest

,Cluster Labels,Asian Restaurant,Indian Restaurant,Location
0,2,0.250000,0.750000,"Andheri,Western Suburbs"
1,5,0.222222,0.777778,"Bandra,Western Suburbs"
2,1,0.000000,1.000000,"Borivali (West),Western Suburbs"
3,1,0.000000,1.000000,"Byculla,South Mumbai"
4,1,0.000000,1.000000,Eastern Suburbs
5,0,0.133333,0.866667,"Ghatkopar,Eastern Suburbs"
6,1,0.000000,1.000000,"Govandi,Harbour Suburbs"
7,1,0.000000,1.000000,Harbour Suburbs
8,1,0.000000,1.000000,"Kalbadevi,South Mumbai"
9,2,0.250000,0.750000,"Kamathipura,South Mumbai"


In [149]:
df=mum_rest.loc[mum_rest['Cluster Labels'] == '0']
df


,Cluster Labels,Asian Restaurant,Indian Restaurant,Location
5,0,0.133333,0.866667,"Ghatkopar,Eastern Suburbs"
12,0,0.111111,0.888889,"Khar,Western Suburbs"


In [150]:
df=mum_rest.loc[mum_rest['Cluster Labels'] == '1']
df

,Cluster Labels,Asian Restaurant,Indian Restaurant,Location
2,1,0.0,1.0,"Borivali (West),Western Suburbs"
3,1,0.0,1.0,"Byculla,South Mumbai"
4,1,0.0,1.0,Eastern Suburbs
6,1,0.0,1.0,"Govandi,Harbour Suburbs"
7,1,0.0,1.0,Harbour Suburbs
8,1,0.0,1.0,"Kalbadevi,South Mumbai"
10,1,0.0,1.0,"Kandivali East,Western Suburbs"
11,1,0.0,1.0,"Kandivali West,Western Suburbs"
13,1,0.0,1.0,"Malad,Western Suburbs"
14,1,0.0,1.0,"Mira-Bhayandar,Western Suburbs"


In [151]:
df=mum_rest.loc[mum_rest['Cluster Labels'] == '2']
df

,Cluster Labels,Asian Restaurant,Indian Restaurant,Location
0,2,0.25,0.75,"Andheri,Western Suburbs"
9,2,0.25,0.75,"Kamathipura,South Mumbai"


In [152]:
df=mum_rest.loc[mum_rest['Cluster Labels'] == '3']
df

,Cluster Labels,Asian Restaurant,Indian Restaurant,Location
17,3,0.181818,0.818182,"Powai,Eastern Suburbs"


In [153]:
df=mum_rest.loc[mum_rest['Cluster Labels'] == '4']
df

,Cluster Labels,Asian Restaurant,Indian Restaurant,Location
19,4,0.042254,0.957746,South Mumbai
22,4,0.058824,0.941176,Western Suburbs


In [154]:
df=mum_rest.loc[mum_rest['Cluster Labels'] == '5']
df

,Cluster Labels,Asian Restaurant,Indian Restaurant,Location
1,5,0.222222,0.777778,"Bandra,Western Suburbs"
